In [ ]:
from brightway2 import *
from bw2regional import *
from bw2_lcimpact import *
from bw2regional.pandarus_remote import AlreadyExists
import rower
import os

In [ ]:
import bw2_lcimpact
bw2_lcimpact.__version__
assert bw2_lcimpact.__version__ >= (0, 3)

In [ ]:
assert rower.__version__ >= (0, 2)

In [ ]:
projects.set_current("LC IMPACT case study")

# Add basic metadata

Such as country definitions

In [ ]:
bw2regionalsetup()

# Fix ecoinvent Rest-of-World definitions

In [ ]:
rower_obj = rower.Rower('ecoinvent')

In [ ]:
rower_obj.load_existing(rower_obj.EI_3_5_CUTOFF)
rower_obj.label_RoWs()

# Fix location labels to include geocollections

In [ ]:
label_activity_geocollections("Fuel comparison")

In [ ]:
label_activity_geocollections("ecoinvent")

Won't be necessary in the next release of `bw2io`

In [ ]:
databases['ecoinvent']['geocollections'] = ['world', 'ecoinvent', 'RoW']
databases['Fuel comparison']['geocollections'] = ['world', 'ecoinvent', 'RoW']
databases['biosphere3']['geocollections'] = []
databases.flush()

# Add regionalized LC IMPACT impact categories

In [ ]:
from bw2_lcimpact.water import *

WaterHumanHealthMarginal().import_regional_method()
WaterEcosystemQualityCertain().import_regional_method()
WaterEcosystemQualityAll().import_regional_method()

In [ ]:
from bw2_lcimpact.pm import *

ParticulateMatterFormationAll().import_regional_method()
ParticulateMatterFormationCertain().import_regional_method()

In [ ]:
from bw2_lcimpact.land_use import *

LandUseOccupationMarginal().import_regional_method()
LandUseOccupationAverage().import_regional_method()
LandUseTransformationMarginalCertain().import_regional_method()
LandUseTransformationMarginalAll().import_regional_method()
LandUseTransformationAverageCertain().import_regional_method()
LandUseTransformationAverageAll().import_regional_method()

In [ ]:
from bw2_lcimpact.climate import *

ClimateChangeHumanHealthCertain100Years("biosphere3").import_global_method()
ClimateChangeHumanHealthAll100Years("biosphere3").import_global_method()
ClimateChangeHumanHealthCertainInfinite("biosphere3").import_global_method()
ClimateChangeHumanHealthAllInfinite("biosphere3").import_global_method()
ClimateChangeTerrestrialEcosystemsCertain100Years("biosphere3").import_global_method()
ClimateChangeTerrestrialEcosystemsAll100Years("biosphere3").import_global_method()
ClimateChangeTerrestrialEcosystemsCertainInfinite("biosphere3").import_global_method()
ClimateChangeTerrestrialEcosystemsAllInfinite("biosphere3").import_global_method()
ClimateChangeAquaticEcosystemsAll100Years("biosphere3").import_global_method()
ClimateChangeAquaticEcosystemsAllInfinite("biosphere3").import_global_method()

# Calculate intersections between inventory and IA spatial scales

In [ ]:
CATEGORIES = [
    "watersheds-hh",
    "watersheds-eq-sw-certain",
    "watersheds-eq-sw-all",
    "particulate-matter",
    "ecoregions",
]

for category in CATEGORIES:
    try:
        remote.calculate_intersection("world", category)
    except AlreadyExists:
        pass

In [ ]:
for category in CATEGORIES:
    remote.intersection("world", category)

## Get third spatial scale

This is the intersection of inventory and IA spatial scales

In [ ]:
for category in CATEGORIES:
    remote.intersection_as_new_geocollection('world', category, 'world-topo-' + category)

In [ ]:
CATEGORIES = [
    "watersheds-hh",
    "watersheds-eq-sw-certain",
    "watersheds-eq-sw-all",
    "particulate-matter",
    "ecoregions",
]

for category in CATEGORIES:
    try:
        remote.calculate_intersection("world-topo-" + category, category)
    except AlreadyExists:
        pass

In [ ]:
for category in CATEGORIES:
    remote.intersection("world-topo-" + category, category)

# Add extension table data

## Add raster `geocollections`

In [ ]:
rasters = [
    'maize_landuse_intensity',
    'maize_water_intensity',
    'sugarcane_landuse_intensity',
    'sugarcane_water_intensity',
    'gdpweighted',
]

for label in rasters:
    fp = os.path.abspath(os.path.join("..", "data", label + ".tif"))
    assert os.path.isfile(fp)

    if label == "gdpweighted":
        label = 'weighted-pop-density'
    
    geocollections[label] = {
        'filepath': fp,
        'band': 1
    }

## Define raster statistics

Need statistics for the spatial scale which is the intersection of inventory and IA

In [ ]:
inters = [
    'world-topo-watersheds-hh',
    'world-topo-watersheds-eq-sw-certain',
    'world-topo-watersheds-eq-sw-all',
    'world-topo-particulate-matter',
    'world-topo-ecoregions',
]

crop_rasters = [
    'maize_landuse_intensity',
    'maize_water_intensity',
    'sugarcane_landuse_intensity',
    'sugarcane_water_intensity',
    'weighted-pop-density',
]

for x in inters:
    for y in crop_rasters:
        try:
            remote.calculate_rasterstats(x, y)
        except AlreadyExists:
            pass

## Import raster statistics as extension tables

In [ ]:
for x in inters:
    for y in crop_rasters:
        remote.rasterstats_as_xt(x, y, x + "-" + y)

In [ ]:
extension_tables